# Generating Minimum Jerk Trajectories (MJT) using CVXPY

This notebook contains the code to generate a set of MJT for a given set of spatiotemporal constraints. The constraints include: movement duration, starting position, target positions, and a set of via points positions and the time at which these via points are reached.

**Note**: To execute this code change the variable "basedrive" to the correct folder addresss.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
import sys
import json
import os

basedrive = "drive/My Drive/0_projects/IMU-Smoothness"
os.chdir(basedrive)
sys.path.append("scripts")

In [0]:
from imupaper.support_mjt import minimum_effort_control_2D
from imupaper.support_mjt import minimum_effort_control_3D
from imupaper.support_mjt import Djerk_matrix
from imupaper.support_mjt import Aeq_matrix
from imupaper.support_mjt import beq_column
from imupaper.support_mjt import plot_trajectory
from imupaper.support_mjt import generate_via_points_2D
from imupaper.support_mjt import generate_via_points_3D
from imupaper.support_mjt import plot_trajectory_3D
from imupaper.support_mjt import extract_all_kinematics_3D_CVXPY
from imupaper.support_mjt import generate_movements_3D

In [0]:
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)

# Generate MJT movements with different via points for virutal IMU
Uses the supprt_mjt module to generate MJT movements with different via points and saves the files to disk.

The trajectories are also plotted and saved in a separate folder.

In [0]:
Nvias = np.arange(0, 11, 1)  # Different no. of via points
Nmoves = [1 if n == 0 else 25 for n in Nvias]
delta = 0.001  # sample rate
t = arange(0, 1, delta)  # one second worth of samples`
Ns = len(t)  # No. of samples.
dim = '3D'

In [0]:
data_dir = f"data/mjtdata"
fig_dir = f"data/mjtdata/figs"
details_file = "data_details.json"
file_fmt = "{0:02d}via_{1:04d}_new"

In [0]:
data_details = {}
for i, nvia in enumerate(Nvias):
    data_details[int(nvia)] = []
    for nmove in range(Nmoves[i]):
        # Display progress
        sys.stdout.write("\r{0:02d} | {1:02d}".format(nvia, nmove))
        sys.stdout.flush()
        
        # Generate via points.
        via_points = generate_via_points_3D(nvia)

        # Generate movement
        prob, move = generate_movements_3D(via_points, Ns, dim)
        
        # Get kinematics and save.
        move_kin = extract_all_kinematics_3D_CVXPY(move, Ns, delta)
        
        # update details
        _fname = file_fmt.format(nvia, nmove)
        _details = {'t': list(via_points[:, 0]),
                    'x': list(via_points[:, 1]),
                    'y': list(via_points[:, 2]),
                    'z': list(via_points[:, 3]),
                    'file': "{0}.csv".format(_fname),
                    'status': prob.status}

        # Generate and save plot
        plot_trajectory_3D(move_kin, via_points, Ns, t,
                           fname="{0}/{1}".format(fig_dir, _fname))
        
        # Save file name
        move_kin.to_csv("{0}/{1}".format(data_dir, _details['file']), index=False)
        
        # Save details files.
        data_details[int(nvia)].append(_details)
        with open("{0}/{1}".format(data_dir, details_file), 'w') as fh:
            json.dump(data_details, fh, indent=4)

10 | 24